In [1]:
# あとで顔検出など使いたくなる可能性あるので、最初からOpenCVでやる
import cv2

import tensorflow as tf
from tensorflow import keras
from keras import layers, models
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Activation, Conv2, Dense, Dropout
from tensorflow.keras.optimizers import Adagrad, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy

from PIL import Image


# データを正方形にリサイズするときの辺の長さ
square_length = 1080


# ラベル付け(0春 1夏 2冬)
# TODO: ? フォルダ名に数字入れておくとかのほうがいいかもしれない？
# TODO: 3つ以上に増やす場合は、class_modeやlossをbinaryからcategricalに変える
def get_label(value):
    if value == "Ariel":
        return 0
    elif value == "Other":
        return 1
    
# ラベル更新したら一緒に更新する
def max_label():
    return 2
    

# 画像の読み込み
datagen = ImageDataGenerator(
    rescale=1./255
#     validation_split=0.1
    )

train_generator = datagen.flow_from_directory(
    "../image/train",
    target_size=(square_length, square_length),
    class_mode='binary',
    batch_size=32,
    subset='training',
)

Found 233 images belonging to 2 classes.


In [ ]:
# モデルの構築
model = keras.Sequential([
    keras.layers.Conv2D(32, (8,8), activation="relu"),
    keras.layers.Conv2D(32, (8,8), activation="relu"),
    keras.layers.MaxPooling2D(pool_size=(4,4)),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(200, activation="relu"),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(1),
    keras.layers.Activation("softmax")
])
# memo: shapeで警告出るのは、keras.layers.reshape((...)),imput_shape=(...))を使えば良さそう


# オプティマイザにAdamを使用
opt = Adam(learning_rate=0.001)

# # モデルをコンパイル
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
# 学習を実行
model.fit(train_generator, steps_per_epoch=1, epochs=20)
print(model.summary())

Epoch 1/20
1/1 [==============================] - 185s 185s/step - loss: 0.6890 - accuracy: 0.6250
Epoch 2/20
1/1 [==============================] - 171s 171s/step - loss: 337.7702 - accuracy: 0.7188
Epoch 3/20
1/1 [==============================] - 181s 181s/step - loss: 117.7887 - accuracy: 0.7188
Epoch 4/20
1/1 [==============================] - 190s 190s/step - loss: 3.9363 - accuracy: 0.7500
Epoch 5/20
1/1 [==============================] - 167s 167s/step - loss: 8.2980 - accuracy: 0.6875
Epoch 6/20
1/1 [==============================] - 162s 162s/step - loss: 4.4145 - accuracy: 0.6562
Epoch 7/20
1/1 [==============================] - 169s 169s/step - loss: 1.2923 - accuracy: 0.7500
Epoch 8/20
1/1 [==============================] - 39s 39s/step - loss: 10.2637 - accuracy: 0.8889
Epoch 9/20
1/1 [==============================] - 150s 150s/step - loss: 29.0153 - accuracy: 0.6562
Epoch 10/20
1/1 [==============================] - 38s 38s/step - loss: 2.3075 - accuracy: 0.6667
Epoch 1

In [6]:
# テスト用ディレクトリの画像でチェック。正解率を表示する。
total = 0.
ok_count = 0.

test_path = "../image/test"
for dir in os.listdir(test_path):
    if dir == ".DS_Store":
        continue

    dir_path = test_path + "/" + dir
    label = get_label(dir)

    for file in os.listdir(dir_path):
        if file != ".DS_Store":
            file_path = dir_path + "/" + file
            image = np.array(Image.open(file_path).resize(size=(square_length, square_length)))
            image = image.transpose(2, 0, 1)
            image = image.reshape(image.shape[0] * image.shape[1] * image.shape[2], 1)
            result = model.predict(np.array([image / 255.]))
            print("label:", label, "result:", result[0])

            total += 1.

            if label == result[0]:
                ok_count += 1.

print("seikai: ", ok_count / total * 100, "%")

label: 0 result: [1.]
label: 0 result: [1.]
label: 0 result: [1.]
label: 0 result: [1.]
label: 1 result: [1.]
label: 1 result: [1.]
label: 1 result: [1.]
label: 1 result: [1.]
seikai:  50.0 %
